In [1]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date
hoje =  pd.to_datetime(date.today())

Current working directory was changed to: C:\Users\viniciusbarros\Documents\GitHub\Previsor\App


In [2]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 12/01/23 - Horário: 12:41:55.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 12/01/23 - Horário: 12:45:24.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 12/01/23 - Horário: 12:45:28.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 12/01/23 - Horário: 12:45:37.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 12/01/23 - Horário: 12:46:07.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 12/01/23 - Horário: 12:48:22.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 12/01/23 - Horário: 12:48:41.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 12/01/23 - Horário: 12:49:47.  Portanto não foi baixado novamente.
vrapeelo

In [3]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
ide_data_cols = ['DthEnvio','IdeUsinaOutorga']
ide_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Baixa bancos de dados

In [4]:
# Carrega BD vmonitoramentoleilao => monitoramentoleilao
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao.sample(5)

,IdeUsinaOutorga,CodLeilao,DatInicioSuprimento
1163,40612,03/2018,2024-01-01
946,37812,04/2019,2025-01-01
1164,40613,04/2019,2025-01-01
153,31439,03/2022,2026-01-01
1197,40733,07/2021,2025-01-01


In [5]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DthEnvio' : 'Data_Envio',
        'NomUsina' : 'usina_nome',
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma.sample(5)

,IdeUsinaOutorga,Data_Envio,canteiroReal,prev_IO,IO_real,DesvRio_real,Ench_Real,IOTrans_Real,Conc_Trans_REAL,Comiss_Real,ME_Real_conc_eol,Conc_Combust_Real,DatRealizacaoVI,DatRealizacaoX,DatRealizacaoV,DatRealizacaoIV
13786,37299,2021-08-03 17:55:27.260,2021-01-19,NaT,2021-01-22,NaT,NaT,2021-02-01,NaT,NaT,NaT,NaT,NaT,NaT,2021-05-13,NaT
9188,44554,2021-03-05 10:39:31.307,2020-09-15,NaT,2020-12-07,NaT,NaT,2021-02-15,NaT,NaT,NaT,NaT,NaT,NaT,2020-12-12,NaT
37801,54654,2022-12-02 15:08:50.037,NaT,2024-10-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
26453,48906,2022-06-03 09:56:52.263,NaT,2022-06-13,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
37018,40691,2022-11-07 09:04:55.660,NaT,2023-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [6]:
# Carrega BD vrapeelcontratorecurso => mercado

cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)

# Carrega BD vrapeellicenciamento => ambiental_datas
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeelacesso => acesso_contratos
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ug_rapeel.loc[ug_rapeel.groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax()]

# Cria dataframe input_rapeel

In [7]:
# Carrega BD vrapeelacesso => acesso_contratos
input_rapeel = pd.merge(mercado,ambiental_datas,on=ide_data_cols,how="inner")
input_rapeel = pd.merge(input_rapeel,acesso_contratos,on=ide_data_cols,how="inner")
input_rapeel = input_rapeel.loc[input_rapeel.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
input_rapeel.sample(5)

,DthEnvio,IdeUsinaOutorga,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT
981,2023-01-03 10:31:02.417,49827,NaT,NaT,2022-12-30,NaT,Não se Aplica,Válido,Não se Aplica,Não Assinado
1186,2023-01-02 13:46:17.857,48588,NaT,2031-07-12,2031-07-12,2031-07-12,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado
2684,2023-01-05 23:27:12.673,47225,NaT,2020-05-22,2020-05-22,2020-05-22,Não se Aplica,Válido,Não se Aplica,Válido
1812,2023-01-04 07:06:29.633,49622,NaT,NaT,2021-03-22,NaT,Não se Aplica,Válido,Não se Aplica,Válido
2619,2023-01-05 19:10:35.397,48642,NaT,NaT,2023-07-27,NaT,Não se Aplica,Não se Aplica,Não se Aplica,Não se Aplica


In [8]:
definicoes =   input_rapeel #pd.merge(input_rapeel,ug_rapeel,on=ide_data_cols,how='inner')
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]
definicoes[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
407,37867,2028-01-11,2026-04-29,2019-09-05,2028-01-11
416,32888,2023-12-30,2022-10-08,2021-10-28,2023-12-30
175,37083,2026-04-19,2025-12-19,2023-06-09,2026-04-19
1516,49479,2030-08-25,2030-08-25,2030-08-25,2030-08-25
123,38325,NaT,2023-08-27,2020-01-12,2023-08-27


In [9]:
cols = ["IdeUsinaOutorga","DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]

monitoramentodados = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]

In [10]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao
rename_cols = {'DatPrevistaComercial':'DatPrevistaComercial'}
definicoes_monitoramento = pd.merge(monitoramentodados,definicoes,on='IdeUsinaOutorga',how="left").rename(columns=rename_cols).drop_duplicates()
definicoes_monitoramento = definicoes_monitoramento.loc[definicoes_monitoramento.IdcUsinaMonitorada == "Sim"]

definicoes_monitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoes_monitoramento.loc[definicoes_monitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()) & (definicoes_monitoramento.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


definicoes_monitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(definicoes_monitoramento[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

definicoes_monitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSitContratoCCD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCCT == nsa) &
    definicoes_monitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoes_monitoramento.loc[
(    ((definicoes_monitoramento.NomSitContratoCCD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCCT == valido)) &
    definicoes_monitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSitContratoCCD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCCT == vencido) |
    (definicoes_monitoramento.NomSitContratoCCD == nass) |
    (definicoes_monitoramento.NomSitContratoCCT == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoes_monitoramento['CondicaoUso'] = "Não informado"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCUST == vencido) |
    (definicoes_monitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoes_monitoramento.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoes_monitoramento["PPA"] = "Ambos"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2020-12-11,2017-09-18,LI,2020-12-11
1,NaT,2026-03-09,2002-12-11,LI,2026-03-09
2,NaT,NaT,NaT,Sem LP,NaT
3,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
7,NaT,NaT,NaT,Não Informado,NaT
...,...,...,...,...,...
3093,NaT,NaT,NaT,Não Informado,NaT
3094,NaT,NaT,NaT,Não Informado,NaT
3095,NaT,NaT,NaT,Não Informado,NaT
3096,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
1241,41901,NaN,NaN,NaT,Não informado
2842,54126,Não se Aplica,Não Assinado,2023-01-05 15:07:02.287,Sem Conexão
410,34186,Válido,Não se Aplica,2023-01-04 23:17:52.887,OK
795,37499,Não se Aplica,Válido,2023-01-04 14:48:38.180,OK
220,32371,Válido,Não se Aplica,2023-01-03 13:41:20.623,OK
2009,48905,Não se Aplica,Não Assinado,2023-01-03 11:24:23.590,Sem Conexão
1421,44565,Não se Aplica,Válido,2023-01-04 16:32:52.977,OK
2427,50206,Não se Aplica,Não Assinado,2023-01-04 17:43:15.623,Sem Conexão
1301,43213,Não se Aplica,Não Assinado,2022-09-01 11:03:11.000,Sem Conexão
2942,54823,Não se Aplica,Válido,2023-01-05 11:37:25.690,OK


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
2323,49861,Não se Aplica,Não Assinado,Sem Conexão
2521,50878,Não se Aplica,Não Assinado,Sem Conexão
2405,50109,Não se Aplica,Não Assinado,Sem Conexão
3061,57216,Não se Aplica,Não Assinado,Sem Conexão
2181,49488,Não se Aplica,Válido,OK
2886,54622,Não se Aplica,Não Assinado,Sem Conexão
2127,49412,Não se Aplica,Não Assinado,Sem Conexão
2194,49501,Não se Aplica,Válido,OK
1995,48759,Não se Aplica,Não Assinado,Sem Conexão
2047,49181,Não se Aplica,Válido,OK


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
595,35526,Fora do ACR,NaT,Nenhum
456,34499,Fora do ACR,2021-07-19,ACL
1831,47377,Fora do ACR,NaT,Nenhum
3090,70555,Fora do ACR,NaT,Nenhum
345,33547,ACR,2021-12-20,Ambos
2534,50945,Fora do ACR,NaT,Nenhum
3000,55381,Fora do ACR,NaT,Nenhum
1775,47307,Fora do ACR,NaT,Nenhum
1054,38128,Fora do ACR,NaT,Nenhum
44,28492,Fora do ACR,NaT,Nenhum


In [11]:
# Apenas mudança de nome
tabela_criterio = definicoes_monitoramento

In [12]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabela_criterio['criterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabela_criterio['dscriterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
134,31271,Paralisada,ACL,LI,OK,8,Obras paralisadas
2635,51691,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
628,35672,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1748,47253,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1473,45012,Em andamento,Ambos,LI,OK,0,Usina em Obras em andamento
1269,43148,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
830,37616,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2242,49649,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1331,44358,Em andamento,Nenhum,LO,OK,0,Usina em Obras em andamento
614,35649,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"


In [13]:
slp = "Sem LP"
sim = "Sim"
tabela_criterio['criterio_novo'] = np.select(
    [
        (tabela_criterio.IdcSemPrevisao == sim),
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        (tabela_criterio.CondicaoAmbiental == "Sem LP"),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
1094,38305,Em andamento,ACR,LI,OK,0,Usina em Obras em andamento
2661,51819,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2860,54155,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2504,50841,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
2842,54126,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"


In [14]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100


In [15]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento ']:
    if 'SigTipoGeracao' in calculo_previsao.columns:
        calculo_previsao.drop(['SigTipoGeracao'],inplace=True,axis=1)

### criterios_da_previsao

In [16]:
criterios_da_previsao  = pd.merge(
                                    tabela_criterio,calculo_previsao,
                                    on="IdeUsinaOutorga",how='inner'
                                    )
criterios_da_previsao = pd.merge(criterios_da_previsao,ug_rapeel,left_on= ['IdeUsinaOutorga','NumUgUsina'],right_on=['IdeUsinaOutorga','NumOperacaoUg'],how='left')

del criterios_da_previsao['NumOperacaoUg']

# Regras

In [17]:
################################ regra_antiga ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regra_antiga'] = pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regra_antiga'] = criterios_da_previsao[['Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regra_antiga'] = ( hoje +pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regra_antiga'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] >= 3) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[criterios_da_previsao['criterio'] >= 3 & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)


################################ regrapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regrapmo'] =  pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regrapmo'] = ( hoje + pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regrapmo'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8) ),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(3*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(4*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5) | (criterios_da_previsao['criterio'] == 7),'regrapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'regrapmo'] = pd.NA

In [18]:
################################ dscjustificativapmo ################################
list_condicoes = [
    (criterios_da_previsao['IdcSemPrevisao'] == "Sim"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"), #& ( criterios_da_previsao['Previsao_OC'] > criterios_da_previsao['DatPrevistaComercial']),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"), #& ( criterios_da_previsao['Previsao_OC'] < criterios_da_previsao['DatPrevistaComercial']),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),
    ( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Dat_OC_obrigacao']),
    ( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Dat_OC_obrigacao']),
    ( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Previsao_OC']),
    ( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Previsao_OC']), 
    (criterios_da_previsao['criterio'] == 3),
    (criterios_da_previsao['criterio'] == 4) | (criterios_da_previsao['criterio'] == 5),
    (criterios_da_previsao['criterio'] == 7),
    (criterios_da_previsao['criterio'] == 6),
]

list_values = [
    "Analisar justificativa",
    "Estágio Atual das Obras",
    "Estágio Atual das Obras",
    "Usina liberada para testes",
    "Testes não concluídos",
    "Paralisação de Obras",
    "Paralisação de Obras",
    "Estágio Atual das Obras",
    "Estágio Atual das Obras" ,
    "Não celebrado contrato de uso do sistema de transmissão/distribuição",
    "Situação da Licença de Instalação",
    "Sem Rapeel",
    "Sem Licenciamento Ambiental"
]

criterios_da_previsao['dscjustificativapmo'] = np.select(list_condicoes,list_values,default="Null")


################################ criterionovopmo ################################
list_condicoes = [
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT")),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT")),
    ((criterios_da_previsao['criterio_novo'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 2)),
    ((criterios_da_previsao['criterio_novo'] == 3)),
    ((criterios_da_previsao['criterio_novo'] == 5)),
    ((criterios_da_previsao['criterio_novo'] == 4)),
    ((criterios_da_previsao['criterio_novo'] == 6)),
    ((criterios_da_previsao['criterio_novo'] == 7)),
    ((criterios_da_previsao['criterio_novo'] == 8)),
    ((criterios_da_previsao['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

criterios_da_previsao['criterionovopmo'] = np.select(list_condicoes,list_values)



criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 9),'dsccriterionovo'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterionovo'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterionovo'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterionovo'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 1),'dsccriterionovo'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 2),'dsccriterionovo'] = "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 3),'dsccriterionovo'] = "Usina sem obras, sem cust, sem PPA e com LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 4),'dsccriterionovo'] = "Usina sem obras, com cust, sem PPA e sem LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 5),'dsccriterionovo'] = "Usina sem obras, sem cust, sem PPA e sem LI = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 7),'dsccriterionovo'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC handicap de 5 anos"

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 6),'dsccriterionovo'] = "Usina sem LP = Previsão OC sem previsão"
criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 8),'dsccriterionovo'] = "Usina obras paralisadas == Previsão OC maior entre data RAPEEL e data compromisso"

In [19]:
################################ regranovapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regranovapmo'] = pd.NA

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].isna()),'regranovapmo'] = criterios_da_previsao['DatPrevistaComercial']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].notna()),'regranovapmo'] = criterios_da_previsao['Previsao_OC']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo']== 1) | (criterios_da_previsao['criterio_novo']== 2) ,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')

criterios_da_previsao.loc[((criterios_da_previsao['criterio_novo'] == 3) | (criterios_da_previsao['criterio_novo'] == 4) | (criterios_da_previsao['criterio_novo'] == 5) ),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 6,'regranovapmo'] = pd.NA

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(6*365,'D')
criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 7,'regranovapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 8,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)


################################ criterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'criterio_pmo_ts'] = 8

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'criterio_pmo_ts'] = 0.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'criterio_pmo_ts'] = 0.2

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'criterio_pmo_ts'] = 0.3

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'criterio_pmo_ts'] = 1.1

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 1.2

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 2.2

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'criterio_pmo_ts'] = 2.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'criterio_pmo_ts'] = 3

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'criterio_pmo_ts'] = 4

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'criterio_pmo_ts'] = 5

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'criterio_pmo_ts'] = 7

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'criterio_pmo_ts'] = 6

In [20]:
################################ dsccriterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias == Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste == Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'dsccriterio_pmo_ts'] = "Usina sem PPA com LI sem CUST = Previsão OC maior entre handicap de 3 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI com CUST = Previsão OC maior entre handicap de 4 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI sem CUST = Previsão OC maior entre handicap de 5 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'dsccriterio_pmo_ts'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC maior entre handicap de 5 anos e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'dsccriterio_pmo_ts'] = "Usina sem LP = Previsão OC sem previsão"

In [21]:
################################ criterio_pmo_ts_max_usina ################################
criterio_pmo_ts_max = criterios_da_previsao.groupby("IdeUsinaOutorga")[['criterio_pmo_ts']].max().rename(columns=
{'criterio_pmo_ts': 'criterio_pmo_ts_max_usina'})
criterios_da_previsao = criterios_da_previsao.merge(criterio_pmo_ts_max,on="IdeUsinaOutorga",how='left')


################################ dscjustificativaregra ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dscjustificativaregra'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['IdcObraParalisada'] == "Sim") & (criterios_da_previsao['IdcSemPrevisao'] != "Sim"),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.2),'dscjustificativaregra'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregra'] = "Estágio das obras da usina"


criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.1),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.2),'dscjustificativaregra'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 2.2),'dscjustificativaregra'] = "Licença de Instalação - LI e acesso a rede - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 3),'dscjustificativaregra'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregra'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregra'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregra'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregra'] = "Sem RAPEEL"


################################ dscjustificativaregranova ################################
criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 9),'dscjustificativaregranova'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 8),'dscjustificativaregranova'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 0.2),'dscjustificativaregranova'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregranova'] = "Estágio das obras da usina"

criterios_da_previsao.loc[(criterios_da_previsao['criterionovopmo'] == 1),'dscjustificativaregranova'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[criterios_da_previsao['criterionovopmo']==2,'dscjustificativaregranova'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[criterios_da_previsao['criterionovopmo']==3,'dscjustificativaregranova'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregranova'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregranova'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregranova'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregranova'] = "Sem RAPEEL"

In [22]:
#criterios_da_previsao_sas = pd.read_excel("./WORK_Criteriosdaprevisao.xlsx")
#save_pickle(criterios_da_previsao_sas,'./criterios_da_previsao_sas.pickle')
#criterios_da_previsao_2_sas = pd.read_excel("./WORK_Criteriosdaprevisao2.xlsx")
#save_pickle(criterios_da_previsao_2_sas,'./criterios_da_previsao_2_sas.pickle')
#tabela_criterio_sas = pd.read_excel("./WORK_TABELCRITERIO.xlsx")
#save_pickle(tabela_criterio_sas,'./tabela_criterio_sas.pickle')

criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga")


criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
criterios_da_previsao["PY"] = True

In [23]:
criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

In [24]:
merge = pd.merge(criterios_da_previsao,criterios_da_previsao_sas,on=['IdeUsinaOutorga','NumUgUsina'],suffixes=('_py','_sas'),how='outer')
merge['OC_diff'] = merge.Previsao_OC - merge.CalculoPrevisorOC_mx

# Teste previsão

In [25]:
merge[['IdeUsinaOutorga','NumUgUsina'] + [ 'Previsao_OC','CalculoPrevisorOC_mx','OC_diff','IdcUsinaMonitorada']].sort_values(by='OC_diff',ascending=False).head(10)

,IdeUsinaOutorga,NumUgUsina,Previsao_OC,CalculoPrevisorOC_mx,OC_diff,IdcUsinaMonitorada
140,30414,1,2025-03-18,2022-09-08,922 days,Sim
146,30612,1,2025-03-29,2023-01-23,796 days,Sim
240,31691,7,2023-08-28,2022-06-05,449 days,Sim
233,31691,5,2023-08-28,2022-06-05,449 days,Sim
234,31691,8,2023-08-28,2022-06-05,449 days,Sim
235,31691,3,2023-08-28,2022-06-05,449 days,Sim
236,31691,6,2023-08-28,2022-06-05,449 days,Sim
238,31691,1,2023-08-28,2022-06-05,449 days,Sim
239,31691,4,2023-08-28,2022-06-05,449 days,Sim
237,31691,2,2023-08-28,2022-06-05,449 days,Sim


# Testes critérios

## criterionovopmo OK

In [26]:
# Check
merge[merge.CriterioNovoPMO != merge.criterionovopmo][['IdeUsinaOutorga','NumUgUsina'] + ['IdcSemPrevisao','flagOPTeste30dias_sas','flagOPTeste30dias_py','FASE_sas','FASE_py','criterio_novo','criterio_sas','criterio_py','CriterioNovoPMO','criterionovopmo']].shape

(6, 12)

## criterio OK

In [27]:
# Check
merge[merge.criterio_sas != merge.criterio_py].shape

(6, 189)

## dscjustificativapmo

In [34]:
merge[merge.DscJustificativaPMO  != merge.dscjustificativapmo][['IdeUsinaOutorga','NumUgUsina'] +['IdcSemPrevisao','IdcObraParalisada_x','FASE_sas','FASE_py','criterio_sas','criterio_py','DatPrevistaComercial','Previsao_OC','CalculoPrevisorOC_mx','DscJustificativaPMO','DatPrevistaComercial','dscjustificativapmo']].sample(5)

,IdeUsinaOutorga,NumUgUsina,IdcSemPrevisao,IdcObraParalisada_x,FASE_sas,FASE_py,criterio_sas,criterio_py,DatPrevistaComercial,Previsao_OC,CalculoPrevisorOC_mx,DscJustificativaPMO,DatPrevistaComercial,dscjustificativapmo
946,32996,7,Não,Não,OUT,OUT,2.0,2,NaT,2024-04-21,2024-03-31,Estágio Atual das Obras,NaT,Null
24412,44989,2,Não,Não,OUT,OUT,1.0,1,2025-07-01,2025-07-01,2025-08-13,Estágio Atual das Obras,2025-07-01,Null
24404,44988,1,Não,Não,OUT,OUT,1.0,1,2025-07-01,2025-07-01,2025-08-13,Estágio Atual das Obras,2025-07-01,Null
634,32483,6,Não,Não,OUT,OUT,1.0,1,NaT,2024-04-24,2024-04-16,Estágio Atual das Obras,NaT,Null
57390,51294,132,Não,Não,OUT,OUT,1.0,1,NaT,2024-06-17,2024-04-09,Estágio Atual das Obras,NaT,Null


In [29]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga","DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada"]
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols]

In [30]:
# vInputCronograma tbm implementado,DATASREAL (LABEL="Idempreendimento" AS Idempreendimento)
cols = ["DatRealizacaoIII","DatRealizacaoV","DatRealizacaoXIV","DatRealizacaoXII","IdeUsinaOutorga","DatRealizacaoIV","DatRealizacaoIX","DatRealizacaoXI","DatRealizacaoX","DthEnvio","DatRealizacaoVI","DatRealizacaoII","CodCeg","DatRealizacaoXIII","DatRealizacaoVII"]
rename_cols = {'DatRealizacaoIII': 'IO_real','DatRealizacaoXIV': 'Conc_Trans_REAL','DatRealizacaoXII': 'Comiss_Real','DatRealizacaoIX': 'DesvRio_real','DatRealizacaoXI': 'Ench_Real','DthEnvio': 'Data_Envio','DatRealizacaoII': 'canteiroReal','DatRealizacaoXIII': 'IOTrans_Real','DatRealizacaoVII': 'ME_Real_conc_eol'}
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates()[cols].rename(columns=rename_cols)
vcronograma["classe"] = vcronograma.CodCeg.str.slice(3,5)
vcronograma["CC_real"] = pd.to_datetime(pd.NA)
vcronograma.loc[vcronograma.classe == "PH","CC_real"] = vcronograma.DatRealizacaoIV
vcronograma.loc[vcronograma.classe == "CV","CC_real"] = vcronograma.DatRealizacaoV
vcronograma["ME_real"] = vcronograma.DatRealizacaoX
vcronograma.loc[vcronograma.classe == "CV","ME_real"] = vcronograma.DatRealizacaoVI